# Guiding with Few-Shot Examples

Hello everyone. Today, we're exploring one of the most reliable techniques for getting structured, predictable output from an LLM. It's based on a classic rule in writing: **Show, don't just tell.** This is the core idea behind **Few-Shot Prompting**.

A **zero-shot** prompt asks the model to perform a task by only *describing* it. A **few-shot** prompt, however, *shows* the model what to do by including a few high-quality examples of the task directly in the prompt. This is a form of "in-context learning" and is incredibly effective for tasks that require a specific output format.

This notebook will demonstrate how to build a few-shot prompt to solve a common developer task: **converting natural language user queries into a structured JSON command.**

## Helper Functions

First, let's set up our reusable helper functions for both `openai` and `litellm`.

In [ ]:
import litellm
import json
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 200

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content

print("Helper functions are defined and ready.")

In [ ]:
user_query = "Can you find my ticket #T-5829?"

## The Zero-Shot Attempt (Telling the Model)

Let's first try to solve this with a zero-shot prompt. We'll describe the JSON format we want in the system prompt and then provide a user query. This approach "tells" the model what to do, but doesn't show it.

In [ ]:
zero_shot_messages = [
    {
        "role": "system",
        "content": dedent("""
            You are a command processing system and your job is to convert a user query
            into a JSON object.
        """)
    },
    {
        "role": "user",
        "content": user_query
    }
]

response_zero_shot = get_completion(
    zero_shot_messages,
    response_format={"type": "json_object"}
)

In [ ]:
"""
{
  "request": {
    "action": "find_ticket",
    "ticket_id": "T-5829"
  }
}
"""

print(response_zero_shot)

## The Few-Shot Solution (Showing the Model)

Now, let's build a **few-shot prompt**. We will provide a sequence of high-quality examples directly in the prompt. For chat models, the correct way to do this is by creating a "conversation" where we act as both the `user` (providing the input) and the `assistant` (providing the correct output).

The model learns the pattern from this conversation.

In [ ]:
few_shot_messages = [
    {
        "role": "system",
        "content": dedent("""
            You are a command processing system and your job is to convert a user query
            into a JSON object.
        """)
    },
    # --- Example 1 ---
    {
        "role": "user",
        "content": "Show me the latest news about Anthropic."
    },
    {
        "role": "assistant",
        "content": json.dumps({ "action": "fetch_news", "params": { "entity": "Anthropic" }})
    },
    # --- Example 2 ---
    {
        "role": "user",
        "content": "What's the weather in Berlin?"
    },
    {
        "role": "assistant",
        "content": json.dumps({ "action": "get_weather", "params": { "entity": "Berlin" }})
    },
    # --- Our new query ---
    {
        "role": "user",
        "content": user_query
    }
]

response_few_shot = get_completion(
    few_shot_messages,
    response_format={"type": "json_object"}
)

In [ ]:
print(response_few_shot)

In [ ]:
business_rules_messages = [
    {
        "role": "system",
        "content": dedent("""
            You are a command processing system and your job is to convert a user query
            into a JSON object.

            Follow the schema:
            {
                "action": <function to call>,
                "params": {
                    "entity": <entity to research>
                }
            }
        """)
    },
    {
        "role": "user",
        "content": user_query
    }
]

response_business_rules = get_completion(
    business_rules_messages,
    response_format={"type": "json_object"}
)

In [ ]:
print(response_business_rules)